In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
path = '../../../../../../Desktop/Orthografietrainer2020/pickle/'

In [75]:
infile = open(path+'sitzungssummary.pkl','rb')
sitzungen = pickle.load(infile)
infile.close()
sitzungen = sitzungen [['ID','UserID','UserAttribut','Art','HA','Fehler']]

sitzungen = sitzungen.dropna()
sitzungen.rename(columns = {'ID':'UebungsID'}, inplace = True)
# Userattribut: 1=Schüler; 0= Gast/ Lehrer
sitzungen['UserAttribut'] = sitzungen['UserAttribut'].replace(['Schüler'],1)
sitzungen['UserAttribut'] = sitzungen['UserAttribut'].replace(['Gast','Lehrer'],0)
sitzungen[['UserAttribut']] = sitzungen[['UserAttribut']].astype('int16')
#HA: bereinigung
sitzungen['HA'] = sitzungen['HA'].replace(['frHA'],'Self')

sitzungen['UebungsID'] = sitzungen['UebungsID'].astype(str)
sitzungen.columns = ['temp_ueb', 'UserID', 'UserAttribut','Art','HA', 'Fehler']


In [76]:
infile = open(path+'schueler2020only.pkl','rb')
schueler = pickle.load(infile)
infile.close()
schueler = schueler[['ID','Geschlecht','Klassenstufe','Anmeldeklassenstufe','Aufgaben','Altaufgaben','done']]

schueler.rename(columns = {'ID':'UserID'}, inplace = True)
schueler['done'] = schueler['done'].astype('int16')
#Geschlecht: bereinigung
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['m.','armin.zekan@hakamste'],'m')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['w.','we'],'w')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['d','wm'],np.nan)
#Klassenstufe: nutze nur Klassenstufe 5-12
options = ['5', '6','7','8','9','10','11','12'] 
schueler = schueler[schueler['Klassenstufe'].isin(options)] 
schueler['Klassenstufe'] = schueler['Klassenstufe'].astype('int')
#Jahredabei: erstelle aus Klassenstufe - Anmeldeklassenstufe die Spalte Jahredabei
schueler = schueler[schueler['Anmeldeklassenstufe'].isin(options)] 
schueler['Anmeldeklassenstufe'] = schueler['Anmeldeklassenstufe'].astype('int')
schueler['Jahredabei'] = schueler['Klassenstufe'] - schueler['Anmeldeklassenstufe']
#Anzahlaufgaben: erstelle Spalte, die angibt, wie viele Aufgaben dem User aktuell zugeordnet sind
schueler['Aufgaben']=schueler['Aufgaben'].str.split()
schueler['AnzahlAufgaben'] = schueler['Aufgaben'].str.len()
schueler['AnzahlAufgaben'] = schueler['AnzahlAufgaben'].replace([np.nan],0)
schueler['AnzahlAufgaben'] = schueler['AnzahlAufgaben'].astype('int')
# Lösche 
schueler= schueler.drop(columns=['Anmeldeklassenstufe','Aufgaben','Altaufgaben'])

In [77]:
infile = open('vacay_preprocessed9.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()

xmlsaetze = xmlsaetze.drop(columns=['group', 'Loesungsnr'])

In [3]:
infile = open('files/predictors_9','rb')
predictors = pickle.load(infile)
infile.close()

predictors['temp_ueb'] = predictors['UebungsID'].map(lambda x: str(x)[:-2])


In [79]:
df = pd.merge(xmlsaetze, predictors, on='UebungsID')
df = pd.merge(df, sitzungen, on='temp_ueb')
df = pd.merge(df, schueler, on='UserID')

In [80]:
def f(row):
    if row.OrderNumber <= 10:
        val = row.OrderNumber -10
    elif row.OrderNumber <=14:
        val = row.OrderNumber -14
    elif row.OrderNumber <=18:
        val = row.OrderNumber -18
    elif row.OrderNumber <=22:
        val = row.OrderNumber -22
    elif row.OrderNumber <=26:
        val = row.OrderNumber -26
    elif row.OrderNumber <=30:
        val = row.OrderNumber -30
    elif row.OrderNumber <=34:
        val = row.OrderNumber -34
    else:
        val = 0
    return val

df['steps'] = df.apply(f, axis=1)

In [81]:
#Sitzungen
dummy_Art = pd.get_dummies(df['Art'], prefix='Art_')
df = pd.merge(left=df, right=dummy_Art,left_index=True,right_index=True)
#HA
dummy_HA = pd.get_dummies(df['HA'], prefix='HA_')
df = pd.merge(left=df, right=dummy_HA,left_index=True,right_index=True)
#Geschlecht
dummy_Geschlecht = pd.get_dummies(df['Geschlecht'], prefix='Sex_')
df = pd.merge(left=df, right=dummy_Geschlecht,left_index=True,right_index=True)
#Testposition
dummy_Testposition = pd.get_dummies(df['Testposition'], prefix='Testposition_')
df = pd.merge(left=df, right=dummy_Testposition,left_index=True,right_index=True)

#Lösche unnötige Spalten
df = df.drop(columns=['Art','HA'])
df= df.drop(columns=['Geschlecht','Testposition'])

In [82]:
df.to_pickle('ready9.pkl')